In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np  
import cv2  
from time import strftime, gmtime
import time
from datetime import datetime,timedelta
import pandas as pd
import shutil
import numpy as np
from tqdm import tqdm, trange
import os

np.set_printoptions(suppress=True)

In [8]:
video_path = "/media/ubuntu/zoro/ubuntu/data/rail/videos_032723"
forward_video_path = video_path + "/101_forward_1/video_101_forward_1_2023-03-27_12_21_19.477906_2023-03-27_12_36_03.677294.mp4"
side_video_path    = video_path +        "/101_side_1/video_101_side_1_2023-03-27_12_21_19.477906_2023-03-27_12_36_03.677294.mp4"
image_path = "/media/ubuntu/zoro/ubuntu/data/rail/data"
image_forward = image_path + "/forward/"
image_side    = image_path + "/side/"

In [9]:
videoCaptureS = cv2.VideoCapture(side_video_path)
size = (int(videoCaptureS.get(cv2.CAP_PROP_FRAME_WIDTH)), int(videoCaptureS.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fps = videoCaptureS.get(cv2.CAP_PROP_FPS) 
frames = videoCaptureS.get(cv2.CAP_PROP_FRAME_COUNT) 
print("帧数："+str(fps))
print("总帧数："+str(frames))
print("视频总时长："+"{0:.2f}".format(frames/fps)+"秒")
strftime("%H:%M:%S", gmtime(frames/fps))

print('--------------------------------') 
videoCaptureF = cv2.VideoCapture(forward_video_path)
size = (int(videoCaptureF.get(cv2.CAP_PROP_FRAME_WIDTH)), int(videoCaptureF.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fps = videoCaptureF.get(cv2.CAP_PROP_FPS) 
frames = videoCaptureF.get(cv2.CAP_PROP_FRAME_COUNT) 
print("帧数："+str(fps))
print("总帧数："+str(frames))
print("视频总时长："+"{0:.2f}".format(frames/fps)+"秒")
strftime("%H:%M:%S", gmtime(frames/fps))

帧数：14.985014985014985
总帧数：11685.0
视频总时长：779.78秒
帧数：15.083333333333334
总帧数：12592.0
视频总时长：834.83秒


'00:13:54'

In [11]:
class Kitti():
    def __init__(self, video_parent_dir, frame_interval, img_width, img_height):
        '''
        :param video_father_dir:  video father dir file dir
        :param frame_interval:   every frame_interval save a key frame
        :param img_width:   img sequence width
        :param img_height:  img sequence height
        '''

        # self.video_dirs = [video_parent_dir+dir for dir in os.listdir(video_parent_dir)]
        self.video_dirs = [video_parent_dir]

        self.frame_interval = frame_interval
        self.img_width = img_width
        self.img_height = img_height
        self.dir_cnt = 0 # 00  文件夹

    def get_format_name(self, idx, lenght):
        '''
        :param idx: given img index, such as 1, 2, 3
        :lenght: format name length
        :return: return format img name like 000001, 000002, ...
        '''
        cnt = lenght - 1
        prefix = ''
        nmb = idx
        while idx // 10 != 0:
            cnt -= 1
            idx = idx // 10
        for i in range(cnt):
            prefix += '0'
        return prefix + str(nmb)
    def run(self,video_dir, save_dir):
        videoCapture = cv2.VideoCapture(video_dir)
        # get video fps
        fps = videoCapture.get(cv2.CAP_PROP_FPS)
        # get vide width and height
        size = (int(videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

        width_offset = int((size[0] - self.img_width) / 2)
        height_offset = int((size[1] - self.img_height) / 2)
        print('=======================')
        print('init_size: ',size)
        print('dest_size: (',self.img_width,',',self.img_height,')')
        img_sequence_parentdir = save_dir + self.get_format_name(self.dir_cnt, 2)
        if not os.path.exists(img_sequence_parentdir):
            os.makedirs(img_sequence_parentdir)
            if not os.path.exists(img_sequence_parentdir + '/image_0/'):
                os.makedirs(img_sequence_parentdir + '/image_0/')
        # 读帧
        success, frame = videoCapture.read()
        total_frame_idx = 0  # video frame index
        count = 0  # keyframe number
        tmp_cnt = 0  # record new cycle frame number
        timestep = 0  # time step
        timestep_total = [0]  # save every time step
        print("Fps: ", fps)
        print('=======================')
        while success:
            if tmp_cnt == 0 or tmp_cnt == self.frame_interval:
                format_img_name = img_sequence_parentdir + '/image_0/' + self.get_format_name(count, 6) + '.png'
                # print(format_img_name)
                cv2.imwrite(format_img_name,
                            frame[height_offset:height_offset + self.img_height,
                            width_offset:width_offset + self.img_width])
                count += 1
                tmp_cnt = 0
                timestep += self.frame_interval / fps # 时间戳, time = 1/f  (s) 单位是s
                timestep_total.append(timestep) 

            success, frame = videoCapture.read()  # 获取下一帧
            total_frame_idx = total_frame_idx + 1
            tmp_cnt += 1
        self.dir_cnt += 1
        np.savetxt(img_sequence_parentdir + '/times.txt', timestep_total)

    def main(self, save_dir):
        for video_dir in self.video_dirs:
            self.run(video_dir, save_dir)

In [30]:
K = Kitti(forward_video_path,1,960,540) # side_video_path
K.main(image_forward) # image_side

init_size:  (1920, 1080)
dest_size: ( 960 , 540 )
Fps:  15.083333333333334


In [12]:
K2 = Kitti(side_video_path,1,720,480) 
K2.main(image_side) 


init_size:  (720, 480)
dest_size: ( 720 , 480 )
Fps:  14.985014985014985


In [ ]:
# 读取遍历 2个数据集,  选择前 1000张,并set interval=5,
# 融合训练bow, 训练 2个 bow: VocF, VocS
# 查询, 
#   if  score(VocF) >0.3 or score(VocS) > 0.3, 判断 为找到目标
#   else  no matching image 

# After completing the above work, establish a station dataset for classification and evaluation and write a description document.